In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
data.head()


In [ ]:
import seaborn as sns

# sns.pairplot(data[['age','diabetes','high_blood_pressure']], diag_kind='kde')
cmap = sns.cubehelix_palette(rot=-.9, as_cmap=True)
sns.relplot(x=data['serum_sodium'], y=data['serum_creatinine'],palette=cmap, size=data['age'], 
            hue=data['creatinine_phosphokinase'], sizes=(10, 100), alpha=.5, height=10)

In [ ]:
sns.pairplot(data, diag_kind='kde')

In [ ]:
sns.pairplot(data[['age','diabetes','platelets','serum_creatinine']], diag_kind='kde', height=5)

In [ ]:
train_stats=data.describe()
train_stats.pop('DEATH_EVENT')
train_stats=train_stats.transpose()
train_stats

In [ ]:
def norm_attribute(x, attribute_name):
    return (x - train_stats['mean'][attribute_name]) / train_stats['std'][attribute_name]

In [ ]:
data['age'] = norm_attribute(data['age'], 'age')
data['platelets'] = norm_attribute(data['platelets'], 'platelets')
data['creatinine_phosphokinase'] = norm_attribute(data['creatinine_phosphokinase'], 'creatinine_phosphokinase')
data['ejection_fraction'] = norm_attribute(data['ejection_fraction'], 'ejection_fraction')
data['serum_creatinine'] = norm_attribute(data['serum_creatinine'], 'serum_creatinine')
data['serum_sodium'] = norm_attribute(data['serum_sodium'], 'serum_sodium')
data['time'] = norm_attribute(data['time'], 'time')
data['anaemia'] = norm_attribute(data['anaemia'], 'anaemia')
data['diabetes'] = norm_attribute(data['diabetes'], 'diabetes')
data['high_blood_pressure'] = norm_attribute(data['high_blood_pressure'], 'high_blood_pressure')
data['sex'] = norm_attribute(data['sex'], 'sex')
data['smoking'] = norm_attribute(data['smoking'], 'smoking')

data.head()

In [ ]:
# get target labels data frame
y_lables = data['DEATH_EVENT']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y_lables, test_size=0.20, random_state=42)

X_train.pop('DEATH_EVENT')
X_test.pop('DEATH_EVENT')

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train

In [ ]:
# def norm(x):
#   return (x - train_stats['mean']) / train_stats['std']
# normed_X_train = norm(X_train)
# normed_X_test = norm(X_test)

In [ ]:
data.describe()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)
# build keras model with optimizer and 3 layer NN
def build_model():
  model = keras.Sequential([
    layers.Dense(300, activation=tf.nn.relu, input_shape=[len(X_train.keys())]),
    layers.Dense(200, activation=tf.nn.relu),
    layers.Dense(1, activation=tf.nn.sigmoid)
  ])

  # optimizer = tf.keras.optimizers.RMSprop(0.05)

  # model.compile(loss='mse',
  #               optimizer=optimizer,
  #               metrics=['mse','mae'])
  
  model.compile(loss=keras.losses.mean_absolute_error,
              optimizer=keras.optimizers.SGD(0.1),
              metrics=['accuracy'])

  return model

In [ ]:
model=build_model()
model.summary()

In [ ]:
keras.utils.plot_model(model, "hd_model.png", show_shapes=True)

In [ ]:
EPOCHS = 300
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
history = model.fit(
    x=X_train, y=y_train, batch_size=None, epochs=EPOCHS, verbose=1,
    callbacks=[early_stop], validation_split=0.2, validation_data=None, shuffle=True,
    class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_batch_size=None, validation_freq=1,
    max_queue_size=10, workers=1, use_multiprocessing=True
)

In [ ]:
history.history.keys()

In [ ]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(20, 10))
plt.grid(True)
# plt.gca().set_ylim(0, 1)
# plt.xticks(np.arange(0, 100, 1))
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_new = X_test[:10]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
y_test[:10]